In [37]:
import pandas_datareader.data as web
import pandas as pd
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import importlib
import plotly.express as px

#Change me!
start = datetime.datetime.today() + relativedelta.relativedelta(years=-5)
split_unit = "6M"
modulename = "BacktestingStrategies.Strategy_RsiOscillator"
classname = "btst.RsiOscillator"
end = datetime.date.today()

data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
data = data.astype("double")

initial = 1000_000
fname = f"html/plot-"

In [38]:
#日付データを検証期間単位ごとにSplit
#https://kakakakakku.hatenablog.com/entry/2021/05/24/002542
grouped_data = data.groupby(pd.Grouper(freq = split_unit))
dfs = [group for _, group in grouped_data]

In [39]:
#検証期間単位ごとにバックテスト＆結果を蓄積
#親ディレクトリ経由で呼び出し   # https://qiita.com/yokohama4580/items/466a483ae022d264c8ee
import os
import sys
sys.path.append(os.pardir)

btst = importlib.import_module(modulename)
# from BacktestingStrategies import Strategy_RsiOscillator as btst
importlib.reload(btst)
import warnings
warnings.simplefilter('ignore')

returns = []

for monthly_data in dfs:
    # バックテストを設定
    bt = Backtest(
        monthly_data, # チャートデータ
        eval(classname), # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    returns.append(output)

In [40]:
import statistics

df = pd.DataFrame(returns)

#最低と最大の期間のBacktestingを表示＆実行
lines = "１期間単位: " + split_unit + "\n"
lines += "テスト期間: " + str(len(returns)) + "\n"
res = round(statistics.mean(df["Return [%]"]), 4)

#リターンが最小のものの期間を取得
min_return = df[df["# Trades"] != 0]["Return [%]"].min()
min_period_start = pd.to_datetime(df[df["Return [%]"] == min_return]["Start"]).values[0]
min_period_end = pd.to_datetime(df[df["Return [%]"] == min_return]["End"]).values[0]
df_min = data[min_period_start : min_period_end]

#リターンが2番目、3番目に小さいものの期間を取得
min_return_3 = df[df["# Trades"] != 0]["Return [%]"].nsmallest(3)
min2_period_start = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[1]]["Start"]).values[0]
min2_period_end = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[1]]["End"]).values[0]
min3_period_start = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[2]]["Start"]).values[0]
min3_period_end = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[2]]["End"]).values[0]
df_min2 = data[min2_period_start : min2_period_end]
df_min3 = data[min3_period_start : min3_period_end]

#リターンが最大のものの期間を取得
max_return = df[df["# Trades"] != 0]["Return [%]"].max()
max_period_start = pd.to_datetime(df[df["Return [%]"] == max_return]["Start"]).values[0]
max_period_end = pd.to_datetime(df[df["Return [%]"] == max_return]["End"]).values[0]
df_max = data[max_period_start : max_period_end]

lines += "平均Return [%]: " + str(res) + "\n"
lines += "最大Return [%]: " + str(max_return) + "\n"
lines += "最大Return期間: " + str(max_period_start) + " - " + str(max_period_end) + "\n"
lines += "最低Return [%]: " + str(min_return) + "\n"
lines += "最低Return期間: " + str(min_period_start) + " - " + str(min_period_end) 

print(lines)



１期間単位: 6M
テスト期間: 11
平均Return [%]: -4.9471
最大Return [%]: 11.32252611
最大Return期間: 2019-04-01T00:00:00.000000000 - 2019-09-30T00:00:00.000000000
最低Return [%]: -36.801341850000014
最低Return期間: 2020-04-01T00:00:00.000000000 - 2020-09-30T00:00:00.000000000


In [41]:
#取引ない月込み
fig = px.box(df["Return [%]"], points = "all")
fig.update_layout(
    xaxis_title = "Strategy",
    yaxis_title = "Return [%]"
)
fig.show()

In [42]:
#最低と最高の期間のバックテストを表示

print("----------WORST----------")
bt = Backtest(
    df_min, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min.html") # 実行結果（グラフ）

print("----------MIN2----------")
bt = Backtest(
    df_min2, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min2.html") # 実行結果（グラフ）

print("----------MIN3----------")
bt = Backtest(
    df_min3, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min3.html") # 実行結果（グラフ）

print("----------BEST----------")
bt = Backtest(
    df_max, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_max.html") # 実行結果（グラフ）

----------WORST----------
Start                     2020-04-01 00:00:00
End                       2020-09-30 00:00:00
Duration                    182 days 00:00:00
Exposure Time [%]                   81.967213
Equity Final [$]                  631986.5815
Equity Peak [$]                     1000000.0
Return [%]                         -36.801342
Buy & Hold Return [%]              -45.884146
Return (Ann.) [%]                  -61.243248
Volatility (Ann.) [%]               14.257169
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -37.165342
Avg. Drawdown [%]                  -37.165342
Max. Drawdown Duration      152 days 00:00:00
Avg. Drawdown Duration      152 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                     -36.817784
Worst Trade [%]                    -36.817784
Avg. Tra

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,910,22,121,1098.41035,694.0,-368013.4185,-0.368178,2020-05-07,2020-09-30,146 days


----------MIN2----------
Start                     2020-10-02 00:00:00
End                       2021-03-31 00:00:00
Duration                    180 days 00:00:00
Exposure Time [%]                   62.295082
Equity Final [$]                  774888.8389
Equity Peak [$]                  1004406.8389
Return [%]                         -22.511116
Buy & Hold Return [%]              -42.243767
Return (Ann.) [%]                  -40.950555
Volatility (Ann.) [%]               18.091922
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -26.566725
Avg. Drawdown [%]                   -14.55582
Max. Drawdown Duration       99 days 00:00:00
Avg. Drawdown Duration       57 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                     -22.522351
Worst Trade [%]                    -22.522351
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1866,46,121,535.63835,415.0,-225111.1611,-0.225224,2020-12-09,2021-03-31,112 days


----------MIN3----------
Start                     2018-04-02 00:00:00
End                       2018-09-28 00:00:00
Duration                    179 days 00:00:00
Exposure Time [%]                   78.461538
Equity Final [$]                 830042.40805
Equity Peak [$]                 1070315.40805
Return [%]                         -16.995759
Buy & Hold Return [%]              -25.256223
Return (Ann.) [%]                  -30.308629
Volatility (Ann.) [%]               16.374175
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -22.448803
Avg. Drawdown [%]                   -11.91536
Max. Drawdown Duration       85 days 00:00:00
Avg. Drawdown Duration       46 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                     -17.009663
Worst Trade [%]                    -17.009663
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,809,28,129,1235.08355,1025.0,-169957.59195,-0.170097,2018-05-10,2018-09-28,141 days


----------BEST----------
Start                     2019-04-01 00:00:00
End                       2019-09-30 00:00:00
Duration                    182 days 00:00:00
Exposure Time [%]                   15.702479
Equity Final [$]                 1113225.2611
Equity Peak [$]                  1118734.1359
Return [%]                          11.322526
Buy & Hold Return [%]               -7.304348
Return (Ann.) [%]                   25.030493
Volatility (Ann.) [%]                11.22694
Sharpe Ratio                         2.229503
Sortino Ratio                        6.724555
Calmar Ratio                         9.645925
Max. Drawdown [%]                   -2.594929
Avg. Drawdown [%]                   -1.306278
Max. Drawdown Duration       13 days 00:00:00
Avg. Drawdown Duration        6 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                      11.880056
Worst Trade [%]                     11.880056
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,877,21,39,1139.6133,1275.0,118734.1359,0.118801,2019-05-08,2019-06-03,26 days


Row(id='29372', ...)